# Marketing Optimization Demo
In this notebook, we will go through a simple demo to show how IBM Decision Optimization will assist the marketing operations to get the best output based on predicted data file 'offers.csv'.

In [1]:
!pip install wget

In [2]:
import wget
url_offer='https://raw.githubusercontent.com/Dylansafool/HandsOn-Notebook/master/offers.csv'

#remove existing files before downloading
!rm -f offer.csv

offer_Filename=wget.download(url_offer)

#list existing files
!ls -l offer.csv

ls: offer.csv: No such file or directory


In [3]:
import pandas as pd

# Read in predicted new offer targeted customer data
offers = pd.read_csv("offers.csv", index_col=0)

offers.head()

,index,age,income,members_in_household,loan_accounts,Savings,Mortgage,Pension,id,nb_products
0,0,38,47958.0,4,1,0,0,0,44256,0
1,1,30,48606.0,2,4,0,0,0,46883,0
2,2,41,42152.0,4,0,0,0,0,32387,0
3,3,42,39788.0,3,3,0,0,0,25504,0
4,4,42,44365.0,6,2,0,1,0,35979,1


Within the CSV file, there are promotion offer predictions for 2756 new customers. The prediction is based on their age, income, members in household and loan account numbers. But the marketing budget is limited. To get the best outcome, a marketing optimization is deployed as follows.

## Preparations
It requires [IBM Watson Studio Cloud](https://www.ibm.com/cloud/watson-studio/>) (Sign up for a [free IBM Cloud account](https://dataplatform.cloud.ibm.com/registration/stepone?context=wdp&apps=all>)
and you can start using this notebook on Watson Studio Cloud right away).

## Step 1 Install IBM docplex package

In [4]:
# install DO package
!pip install docplex

In [5]:
# import related module
import sys
import docplex.mp

# ingore the warnings
import warnings
warnings.filterwarnings('ignore')

## Step 2 Problem Description

* For this case, we have a total of \$25000 marketing budget. 

* For the 2756 target customers, there are three products we want to sell: 'Saving', 'Mortgage' and 'Pension'. Each product has its own revenue.

* We have three marketing channels ('gift', 'newsletter' and 'seminar'). Each channel has its own cost per customer and successful rate. 

* From the previous predictive analysis, we have collected the data and known which product is the best to offer to each customer. 

Since the budget is limited, Marketing offers need to be decided in this way:

* Every Channel has at least 10\% of the total offers 
* At most 1 product is offered to each customer
* Total offer cost should be within the budget

In [6]:
# Total available budget
availableBudget = 25000

# Three products to sell
products = ["Savings", "Mortgage", "Pension"]
# How much revenue is earned when selling each product
productValue = [200, 300, 400]
value_per_product = {products[i] : productValue[i] for i in range(len(products))}

# Three marketing channels with cost and successful rate information
channels =  pd.DataFrame(data=[("gift", 20.0, 0.20), 
                               ("newsletter", 15.0, 0.05), 
                               ("seminar", 23.0, 0.30)], columns=["name", "cost", "factor"])

# get the range for offers, products and channels
offersR = range(0, len(offers))
productsR = range(0, len(products))
channelsR = range(0, len(channels))

In [7]:
channels

,name,cost,factor
0,gift,20.0,0.20
1,newsletter,15.0,0.05
2,seminar,23.0,0.30


## Step 3 Setup URL and Key from Decision Optimization on cloud
After sign-up of IBM cloud account, login and go to this website:
https://dropsolve-oaas.docloud.ibmcloud.com/dropsolve/api to get API key and base URL.

### !!!Change the URL and Key, otherwise this notebook can not run!!!

In [8]:
# setup IBM Decision Optimization environment
url = "https://api-oaas.docloud.ibmcloud.com/job_manager/rest/v1/"
key = "api_c9c32757-0856-493b-b924-5b8155c384c0"

from docplex.mp.context import Context
context = Context.make_default_context()

context.solver.docloud.url = url
context.solver.docloud.key = key
context.solver.agent = 'docloud'

## Step 4 Starting build up an optimization model

### Starting the Decision Optimization environment

In [9]:
# Starting a Decision Optimization model
from docplex.mp.model import Model

mdl = Model(name="marketing_campaign", checker='on', context=context)

### Define the decision varibales ( Choices of actions)

In [10]:
# These varibales are binary with three dimesion (offer, product, channel)
# binary means the value is either 0 or 1
# Variable X(offer_i, product_j, channel_k) = 1 means for customer i, use channel k to promote product j
# otherwise the variable is 0

channelVars = mdl.binary_var_cube(offersR, productsR, channelsR)

### Define the constrains (Limits of the reality)

In [11]:
# At most 1 product is offered to each customer
mdl.add_constraints( mdl.sum(channelVars[o,p,c] for p in productsR for c in channelsR) <=1 for o in offersR)

# Do not exceed the budget
mdl.add_constraint( mdl.sum(channelVars[o,p,c]*channels.get_value(index=c, col="cost") 
                                           for o in offersR 
                                           for p in productsR 
                                           for c in channelsR)  <= availableBudget, "budget")  

# At least 10% offers per channel
for c in channelsR:
    mdl.add_constraint(mdl.sum(channelVars[o,p,c] for p in productsR for o in offersR) >= len(offers) // 10)

    
# print the information of this model
mdl.print_information()

Model: marketing_campaign
 - number of variables: 24804
   - binary=24804, integer=0, continuous=0
 - number of constraints: 2760
   - linear=2760
 - parameters: defaults


### Setup the objective value (Target)
Max the products revenue with current predicted data

In [12]:
# for each offer action, the coefficient = successful rate(factor)* product revenue
# sum the action*coefficient as the objective vaule and maximize it
obj = 0

for c in channelsR:
    for p in productsR:
        product=products[p]
        coef = channels.get_value(index=c, col="factor") * value_per_product[product]
        obj += mdl.sum(channelVars[o,p,c] * coef* offers.get_value(index=o, col=product) for o in offersR)

mdl.maximize(obj)

### Solve the model and get the results

In [13]:
# set the time limit for solving the model
mdl.parameters.timelimit = 30

# solve the model
s = mdl.solve()

# make sure it has a solution
assert s, "No Solution !!!"

# print the solution
print(mdl.get_solve_status())
print(mdl.get_solve_details())

JobSolveStatus.OPTIMAL_SOLUTION
status  = integer optimal solution
time    = 0.945038 s.
problem = MILP
gap     = 0%



## Step 5 Analyze the solution
Counting the offer numbers for each productions and each channel and save as KPI

In [14]:
# setup KPI for the model
totaloffers = mdl.sum(channelVars[o,p,c] 
                      for o in offersR
                      for p in productsR 
                      for c in channelsR)
mdl.add_kpi(totaloffers, "nb_offers")

budgetSpent = mdl.sum(channelVars[o,p,c]*channels.get_value(index=c, col="cost") 
                                           for o in offersR 
                                           for p in productsR 
                                           for c in channelsR)
mdl.add_kpi(budgetSpent, "budgetSpent")

for c in channelsR:
    channel = channels.get_value(index=c, col="name")
    kpi = mdl.sum(channelVars[o,p,c] for p in productsR for o in offersR)
    mdl.add_kpi(kpi, channel)

for p in productsR:
    product = products[p]
    kpi = mdl.sum(channelVars[o,p,c] for c in channelsR for o in offersR)
    mdl.add_kpi(kpi, product)

In [15]:
# print out the KPI and objective value
mdl.report()

* model marketing_campaign solved with objective = 72620.000
*  KPI: nb_offers   = 1218.000
*  KPI: budgetSpent = 24989.000
*  KPI: gift        = 275.000
*  KPI: newsletter  = 275.000
*  KPI: seminar     = 668.000
*  KPI: Savings     = 693.000
*  KPI: Mortgage    = 381.000
*  KPI: Pension     = 144.000


### Translate the none zero solutions to offer details and save in text file

In [16]:
# Save the offers to act in a text file
Result_actions = []
for o in offersR:
    for p in productsR: 
        for c in channelsR:
            if channelVars[o,p,c].solution_value >= 0.5:
                result_line = ('Offer '+str(o)+', ' 
                               + channels.get_value(index=c, col="name")
                               + ', ' + products[p]
                              )
                Result_actions.append(result_line)

In [17]:
with open('Actions_To_Do.txt', 'w') as f:
    for item in Result_actions:
        f.write("%s\n" % item)